In [1]:
from PIL import Image
from pillow_lut import load_cube_file
import os
import random
import random
from datetime import datetime
import numpy as np
import cv2
random.seed(datetime.now().timestamp())

In [2]:


luts = []

for lut in os.listdir("./LUTS/"):
    luts.append(load_cube_file(os.path.join("./LUTS/", lut)))

for images in os.listdir("./data/Teal&Orange/origin_jpgs/"):
    lut = luts[1]
    img = Image.open(os.path.join("./data/Teal&Orange/origin_jpgs/", images))
    im = cv2.imread(os.path.join("./data/Teal&Orange/origin_jpgs/", images))
    # print(images)
    # print(im.shape)
    if im.shape[0] > im.shape[1]:
        img_ = img.convert('RGBA').rotate(90, expand=1)
        # print(1)
        fff = Image.new("RGBA", img_.size, (255,255,255,255))
        out = Image.composite(img_, fff, mask=img_)
        out.convert(img.mode).save(os.path.join("./data/Teal&Orange/origin_jpgs/", images))
        img = Image.open(os.path.join("./data/Teal&Orange/origin_jpgs/", images))
    img.filter(lut).save(os.path.join("./data/Teal&Orange/new_jpgs/", images))


In [3]:
import skimage.transform as trans


def train_or_test():
    x = random.random()
    return 1 if x >= 0.2 else 0


def random_crop(img):
    h,w = img.shape[0], img.shape[1]
    delta_h = (h // 480) * 480
    delta_w = (w // 640) * 640
    h_st = random.randint(0, h - delta_h)
    w_st = random.randint(0,w - delta_w)

    crop_img = img[h_st:h_st+delta_h, w_st:w_st + delta_w,:]

    return h_st, w_st, delta_h, delta_w, trans.downscale_local_mean(crop_img, (delta_h // 480,delta_w // 640,1))

# a = np.zeros((4000,6000,3))
# print(trans.downscale_local_mean(a, (a.shape[0] // 480, a.shape[1] // 960,1)).shape)



In [4]:
train_list = []
test_list = []
for image in os.listdir("./data/Teal&Orange/origin_jpgs/"):
    ori_img_pth = os.path.join("./data/Teal&Orange/origin_jpgs/", image)
    new_img_pth = os.path.join("./data/Teal&Orange/new_jpgs/", image)
    ori_img = cv2.imread(ori_img_pth)
    new_img = cv2.imread(new_img_pth)

    # print(ori_img.shape)
    # print(new_img.shape)
    # if(ori_img.shape[0] == 480):
    #     continue
    for i in range(1):
        h_st, w_st, d_h, d_w, img_ = random_crop(ori_img)
        cv2.imwrite(img = img_, filename= ori_img_pth.replace(".JPG", "_" + str(i) + ".JPG"))
        cv2.imwrite(img = trans.downscale_local_mean(new_img[h_st:h_st+d_h, w_st:w_st + d_w,:], (d_h // 480,d_w // 640,1)), filename= new_img_pth.replace(".JPG", "_" + str(i) + ".JPG"))
    
    if train_or_test() > 0:
        for i in range(1):
            train_list.append(image.replace(".JPG", "_" + str(i) + ".JPG"))
    else:
        for i in range(1):
            test_list.append(image.replace(".JPG", "_" + str(i) + ".JPG"))


 


In [8]:
print(len(train_list))
print(len(test_list))

396
103


In [9]:
with open("./data/Teal&Orange/train_img.txt", "w") as f:
    for image in train_list:
        f.write(image+'\n')


with open("./data/Teal&Orange/test_img.txt", "w") as f:
    for image in test_list:
        f.write(image+'\n')

In [7]:
import rawpy

